In [22]:
import os
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

True

## Loading LLM
We will load OpenAI GPT-4 LLM to assist our agents

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview")

## Import Modules Needed for this Project

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.schema import Document
from typing_extensions import TypedDict
from typing import List
import json

## Let us create some Utility function

In [5]:
def write_python_file(content, filename):
    """Writes the given content as a python file to the local directory.

    Args:
        content: The string content to write to the file.
        filename: The filename to save the file as.
    """
    # Get the directory of the current script
    script_dir = os.path.dirname(os.path.realpath(__file__))
    
    # Construct the full path to the file
    file_path = os.path.join(script_dir, 'src', f"{filename}.py")
    
    # Ensure the 'src' directory exists, if not, create it
    src_dir = os.path.join(script_dir, 'src')
    if not os.path.exists(src_dir):
        os.makedirs(src_dir)
    
    # Write the content to the file
    with open(file_path, "w") as f:
        f.write(content)

In [8]:
def read_input_json(file_path) -> str:
    """Reads JSON data from a file and returns it as a string.

    Args:
        file_path: The path to the JSON file.

    Returns:
        A string representation of the JSON data.
    """
    with open(file_path, 'r') as user_input_file:
        data = json.load(user_input_file)
    
    user_input = json.dumps(data)

    return user_input


## Create a Generaric funtion to create an Agent

In [12]:
# To dynamically create any number of agents
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_core.messages import (
    BaseMessage,
    ToolMessage,
    HumanMessage,
)
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph

# def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
#     # Define the prompt using LCEL
#     prompt = ChatPromptTemplate.from_messages([
#         ("system", system_prompt),
#         MessagesPlaceholder(variable_name="messages"),
#         MessagesPlaceholder(variable_name="agent_scratchpad"),
#     ])
    
#     # Create the agent using LCEL
#     agent = (
#         {
#             "messages": lambda x: x["messages"],
#             "agent_scratchpad": lambda x: x["agent_scratchpad"],
#         }
#         | prompt
#         | llm
#         | StrOutputParser()
#     )
    
#     return agent
def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards completing the task."
                " If you are unable to fully Complete the task, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    if not tools:
        prompt = prompt.partial(tool_names="")
        return prompt | llm
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

## Define tools to be used by Agents

In [ ]:
from langchain_core.tools import tool, StructuredTool
from typing import Annotated
import os

PROJECT_PATH = "Projects"

@tool
def write_generated_code_to_file(
    generated_code: Annotated[str, "The code generated by the agent."],
    file_path: Annotated[str, "The path where the generated code should be written."],
    project_folder: Annotated[str, "The name of the project where all files needed for the project are to placed."]
):
    """
    Writes the provided generated code to the specified file within the project structure.

    Args:
        generated_code (str): The code generated by the agent.
        file_path (str): The path where the generated code should be written.
    """
    try:
        # Prepend PROJECT_PATH to the file_path to ensure all projects are written to the PROJECT_PATH parent folder
        parent_dir_path= os.path.dirname(PROJECT_PATH)
        full_path = os.path.join(parent_dir_path, project_folder, file_path)

        # Ensure the directory exists before writing the file
        os.makedirs(os.path.dirname(full_path), exist_ok=True)
        
        # Write the generated code to the file
        with open(full_path, 'w') as file:
            file.write(generated_code)
        
        return f"Successfully wrote generated code to {full_path}."
    except BaseException as e:
        return f"Failed to write generated code. Error: {repr(e)}"
    

In [ ]:
# writer_tool = StructuredTool.from_function(write_generated_code_to_file)

## Define State

In [ ]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

## Define Agent Nodes

In [ ]:
import functools
from langchain_core.messages import AIMessage
from langchain.agents import AgentType


# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }

# architect agent and node
Solution_Architect = create_agent(
    llm,
    [],
    system_message="You are a requirements specialist and you should break down requirement given to you into well defined deliverables and provide them to the coder one task at a time.",
)
architect_node = functools.partial(agent_node, agent=Solution_Architect, name="Architect")

coder_agent = create_agent(
    llm,
    [write_generated_code_to_file],
    system_message="Source files generated by you are written to the local directory",
)
coder_node = functools.partial(agent_node, agent=coder_agent, name="Coder")

## Define a Tool Node

In [ ]:
from langgraph.prebuilt import ToolNode

tools = [write_generated_code_to_file]
tool_node = ToolNode(tools)

## Define Edge Logic

In [ ]:
# Either agent can decide to end
from typing import Literal


def router(state) -> Literal["call_tool", "__end__", "continue"]:
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        # The previus agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "__end__"
    return "continue"

## Define the Graph

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("Architect", architect_node)
workflow.add_node("Coder", coder_node)
workflow.add_node("call_tool", tool_node)

workflow.add_conditional_edges(
    "Architect",
    router,
    {"continue": "Coder", "call_tool": "call_tool", "__end__": END},
)
workflow.add_conditional_edges(
    "Coder",
    router,
    {"continue": "Architect", "call_tool": "call_tool", "__end__": END},
)

workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "Architect": "Architect",
        "Coder": "Coder",
    },
)
workflow.set_entry_point("Architect")
graph = workflow.compile()

## Invoke the Graph

In [ ]:
user_input = read_input_json("test.json")
pp.pp(user_input)

events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content=f"Create this project for me. Requirements are {user_input}. Once you code it up, finish."
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 100},
)
for s in events:
    print(s)
    print("----")

# Everything below experimentation

## Let's Create Basic chains needed by our agents

In [13]:
# # Requirements Document Generator
# supervisor_prompt = PromptTemplate(
#     template="""You are a requirements gathering specialist and your job is to understand the user requirement from the json text {json_user_input} and create a detailed, \
#         well defined requirements prompt from it that could be broken down into individual tasks to assign to team members!""", input_variables=["json_user_input"],)

# supervisor_prompt = PromptTemplate(template="""You are excellent at system design and task break down, Break this task {summary} precisely and delegate the deliverables with precise requirements to different {members} to complete it independently.""", input_variables=["summary","members"])

# coder_prompt = PromptTemplate(template="""You are expert python developer, complete this coding {task} following all the security standards, secure packages, error handling, naming conventions and code documentation and then write it to the project folder""", input_variables=["task"])


In [43]:
# Supervisor Chain
members = ["SystemArchitect", "Coder", "Tester", "DeploymentSpecialist"]
supervisor_prompt = PromptTemplate(
    template = """You are a requirements gathering specialist and your job is to understand the user requirement from the json text {json_user_input} and create a very detailed, \
        well defined requirements prompt from it that could be forwarded to solution architect to break down into individual deliverables.! 
        """,
        input_variables=["json_user_input"],
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

supervisor_chain = supervisor_prompt | llm | StrOutputParser()

json_user_input = read_input_json("test.json")
members_str = ','.join(members)

print(json_user_input)
print(members_str)


result = supervisor_chain.invoke({"json_user_input":json_user_input})

print(result)

{"edges": [], "nodes": [{"id": "node-b0", "name": "student-service", "language": "python", "grpcConfig": {"server": {"sqlDB": "Map", "port": "50052", "resources": [{"fields": {"Name": {"datatype": "str"}, "RollNumber": {"datatype": "int"}, "College": {"datatype": "str"}, "Sign": {"datatype": "str"}, "Marks": {"datatype": "int"}, "GateScore": {"datatype": "int"}, "IsPassed": {"datatype": "bool"}}, "name": "Student"}]}, "framework": "grpcio"}}]}
SystemArchitect,Coder,Tester,DeploymentSpecialist
Project Overview:
The project is centered around the development of a microservice called "student-service", which will be written in Python and utilize the gRPC framework (specifically grpcio) for communication. This microservice will be responsible for handling student-related data and operations.

Functional Requirements:

1. Microservice Identification:
   - The microservice shall be identified as "student-service".

2. Programming Language:
   - The microservice shall be implemented using Pyt

In [45]:
# Solution Architect Chain
architect_prompt = PromptTemplate(template = """You are excellent at system design and task break down, Break this task {summary} precisely into independent tasks\
                                  and delegate the deliverables with precise requirements to the expert programmer in your team {members_str} to complete it task by task. Make sure the response follows a json format for easy parsing""", input_variables=["summary","members_str"]
                                  )

architect_chain = architect_prompt | llm | JsonOutputParser()

summary = result
new_result = architect_chain.invoke({"summary":summary, "members_str":members_str})

print(new_result)

{'ProjectTasks': [{'TaskID': 1, 'Title': 'Review and Approval of Requirements Document', 'Description': 'Review the project requirements document with stakeholders for approval before starting the technical design.', 'AssignedTo': 'SystemArchitect', 'DueDate': 'YYYY-MM-DD', 'Dependencies': []}, {'TaskID': 2, 'Title': 'Microservice Design Document Creation', 'Description': "Create a detailed design document outlining the architecture, components, data models, gRPC service definitions, and interaction flows for 'student-service'.", 'AssignedTo': 'SystemArchitect', 'DueDate': 'YYYY-MM-DD', 'Dependencies': ['TaskID: 1']}, {'TaskID': 3, 'Title': 'Implementation Plan Development', 'Description': 'Develop a step-by-step implementation plan that includes development, testing, and deployment phases.', 'AssignedTo': 'SystemArchitect', 'DueDate': 'YYYY-MM-DD', 'Dependencies': ['TaskID: 2']}, {'TaskID': 4, 'Title': 'Microservice Source Code Development', 'Description': "Program the 'student-servic

In [47]:
import pprint as pp
pp.pp(new_result)

{'ProjectTasks': [{'TaskID': 1,
                   'Title': 'Review and Approval of Requirements Document',
                   'Description': 'Review the project requirements document '
                                  'with stakeholders for approval before '
                                  'starting the technical design.',
                   'AssignedTo': 'SystemArchitect',
                   'DueDate': 'YYYY-MM-DD',
                   'Dependencies': []},
                  {'TaskID': 2,
                   'Title': 'Microservice Design Document Creation',
                   'Description': 'Create a detailed design document outlining '
                                  'the architecture, components, data models, '
                                  'gRPC service definitions, and interaction '
                                  "flows for 'student-service'.",
                   'AssignedTo': 'SystemArchitect',
                   'DueDate': 'YYYY-MM-DD',
                   'Dependencies

In [39]:
# Construct the options part
options_str = ", ".join(options)
print(options_str)

# Construct the members part
members_str = ", ".join(members)
print(members_str)

FINISH, SystemArchitect, Coder, Tester, DeploymentSpecialist
SystemArchitect, Coder, Tester, DeploymentSpecialist


## Define the Agent State Class that will be passed to other agents in the graph

In [14]:
from typing import TypedDict, List, Union, Dict, Any

class AgentState(TypedDict):
    user_input: str
    current_task: str
    tasks: List[str]
    worker_results: Dict[str, Any]

## Define Supervisor and worker agent node initilazation

In [15]:
def supervisor_agent(state: AgentState, filepath: str) -> AgentState:
    # Initial state setup
    if not state.get('user_input'):
        state['user_input'] = read_input_json(filepath)
        state['current_task'] = 'Convert user input to requirements prompt'
    elif state['current_task'] == 'Convert user input to requirements prompt':
        # Assuming the worker agent has processed the input and returned a requirements prompt
        state['current_task'] = 'Break down requirements into tasks'
    elif state['current_task'] == 'Break down requirements into tasks':
        # Assuming the worker agent has broken down the requirements into tasks
        state['current_task'] = 'Assign tasks to workers'
    return state

In [16]:
def worker_agent(state: AgentState) -> AgentState:
    if state['current_task'] == 'Convert user input to requirements prompt':
        # Simulate converting user input to a requirements prompt
        state['current_task'] = 'Break down requirements into tasks'
        state['tasks'] = ['Task 1', 'Task 2', 'Task 3']
    elif state['current_task'] == 'Break down requirements into tasks':
        # Simulate breaking down the requirements into tasks
        state['current_task'] = 'Assign tasks to workers'
        state['worker_results'] = {'A': ['Task 1'], 'B': ['Task 2'], 'C': ['Task 3']}
    return state

## Now Let's build a simple langgraph and see how this setup works

In [33]:
from langgraph.graph import StateGraph, END
import json

members = ["SystemArchitect", "Coder", "Tester", "DeploymentSpecialist"]
json_user_input = read_input_json("test.json")
# Initialize the state with default values
initial_state = {
    "user_input": None, # Assuming an empty string as default for user_input
    "current_task": "", # Assuming an empty string as default for current_task
    "tasks": [], # Assuming an empty list for tasks
    "worker_results": {} # Assuming an empty dictionary for worker_results
}


# Create the supervisor agent
supervisor_agent_instance = create_agent(llm, [], f"You are a requirements gathering specialist and your job is to understand the user requirement from the json text {json_user_input} and create a detailed, \
        well defined requirements prompt from it that could be broken down into individual tasks to assign to team members!")

# Create the worker agent
worker_agent_instance = create_agent(llm, [], "You are an efficient worker who completes the tasks to perforfection.")

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes for the supervisor and worker agents
workflow.add_node("supervisor", supervisor_agent(state=initial_state, filepath = "test.json"))
workflow.add_node("worker", worker_agent)

# Set the entry point as `supervisor`
workflow.set_entry_point("supervisor")

# Add conditional edges to control the flow
workflow.add_conditional_edges(
    "supervisor",
    lambda state: state['current_task'],
    {
        "Convert user input to requirements prompt": "worker",
        "Break down requirements into tasks": "worker",
        "Assign tasks to workers": END
    }
)

# Add an edge from `worker` back to `supervisor`
workflow.add_edge('worker', 'supervisor')

# Compile the graph
app = workflow.compile()

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

## Let's initialize the AgentState and run the graph to see how it works

In [30]:
# Invoking the graph to see how it performs.
for s in app.stream(
    initial_state
):
    if "__end__" not in s:
        print(s)
        print("----")

TypeError: supervisor_agent() missing 1 required positional argument: 'filepath'